# Eagle Eye

In this notebook the relevant test and calibrations for the eagle eye were done. This calibration is based on the images with straight lines:


![Lanes Image](../test_images/straight_lines1.jpg) 

![Lanes Image](../test_images/straight_lines2.jpg) 


In [11]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from pathlib import Path
import os
from pipeline import undistort_image

In [12]:
straight_lines1_file =   "../test_images/straight_lines1.jpg"
straight_lines2_file =   "../test_images/straight_lines2.jpg"


straight_lines1 = mpimg.imread(straight_lines1_file)
straight_lines2 = mpimg.imread(straight_lines2_file)

# Undistorting image
straight_lines2 = undistort_image(straight_lines2)
straight_lines1 = undistort_image(straight_lines1)

def plot_corners():
    plt.plot(293, 670, '.')
    plt.plot(1018, 670, '.')
    plt.plot(577, 466, '.')
    plt.plot(707, 466, '.')

# Visualy selection of 4 points to make a rectangle
plt.figure(1)
pts = np.array(
    [[293, 670],
     [577, 463],
     [704, 463],
     [1018, 670]], np.int32)
pts = pts.reshape((-1,1,2))
cv2.polylines(straight_lines1,[pts],True,(0,255,255), 2)
plt.imshow(straight_lines1)
plot_corners()

plt.figure(2)
# Visualy selection of 4 points to make a rectangle
cv2.polylines(straight_lines2,[pts],True,(0,255,255), 2)
plt.imshow(straight_lines2)
plot_corners()

It was observed that this 4 points fits both images quite nicely so we will use them as starting points to select the corners of the perspective transformation. But I tried to tuned the point to produce the straightest lines simultaneously on both images. It was some kind of trade off, because while making one image straighter, the other one bends. Finally I arrive to the values shown below which make both look decent.

In [13]:
src = np.float32(
    [[293, 670],
     [1018, 670],
     [577, 463],
     [704, 463]])

dst = np.float32(
    [[293, 670],
     [1018, 670],
     [293, 463],
     [1018, 463]])

mtx_perspective = cv2.getPerspectiveTransform(src, dst)
mtx_inv_perspective = cv2.getPerspectiveTransform(dst, src)

img_size = (straight_lines1.shape[1],straight_lines1.shape[0])

straight_lines1_warped = cv2.warpPerspective(straight_lines1, mtx_perspective, img_size, flags=cv2.INTER_LINEAR)
straight_lines2_warped = cv2.warpPerspective(straight_lines2, mtx_perspective, img_size, flags=cv2.INTER_LINEAR)


output_dir = "../output_images/eagle_eye/"
Path(output_dir).mkdir(parents=True, exist_ok=True)

f1 = plt.figure(3)
plt.imshow(straight_lines1_warped)
f1.savefig(output_dir + "straight_lines1.jpg")

f2 = plt.figure(4)
plt.imshow(straight_lines2_warped)
f2.savefig(output_dir + "straight_lines2.jpg")

np.save("mtx_perspective", mtx_perspective)
np.save("mtx_inv_perspective", mtx_inv_perspective)

The two straight lines in eagle look as follows:

![Lanes Image](../output_images/eagle_eye/straight_lines1.jpg) 

![Lanes Image](../output_images/eagle_eye/straight_lines2.jpg) 

In [14]:
# test pipeline implementation

from pipeline import eagle_eye, eagle_eye_inv

straight_lines1_warped_test = eagle_eye(straight_lines1)

plt.figure()
plt.imshow(straight_lines1_warped_test)

straight_lines1_unwarped_test = eagle_eye_inv(straight_lines1_warped_test)

plt.figure()
plt.imshow(straight_lines1_unwarped_test)